In [1]:
import os
import math
import torch
import wandb
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from lightning.pytorch.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.utilities.model_summary import ModelSummary
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryRecall, BinaryPrecisionRecallCurve
from torchvision import transforms
from torchinfo import summary
import timm

import sklearn
import numpy as np

import matplotlib.pyplot as plt

from data.datamodule import Animal_DataModule

from data.cats_and_dogs import BinaryCIFARDataModule

c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading Configuration

In the following steps, we will load the configuration settings using the `load_configuration` function. The configuration is stored in the `config` variable which will be used throughout the script.

In [2]:
from config.load_configuration import load_configuration
config = load_configuration()

PC Name: DESKTOP-LUKAS
Loaded configuration from config/config_lukas.yaml


### Logging in to Weights & Biases (wandb)

Before starting any experiment tracking, ensure you are logged in to your Weights & Biases (wandb) account. This enables automatic logging of metrics, model checkpoints, and experiment configurations. The following code logs you in to wandb:

```python
wandb.login()
```
If you are running this for the first time, you may be prompted to enter your API key.

In [3]:
wandb.login()

wandb: Currently logged in as: lukas-pelz (HKA-EKG-Signalverarbeitung) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

### Setting Seeds for Reproducibility

To ensure comparable and reproducible results, we set the random seed using the `seed_everything` function from PyTorch Lightning. This helps in achieving consistent behavior across multiple runs of the notebook.

In [4]:
pl.seed_everything(config['seed'])
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"   # disable oneDNN optimizations for reproducibility

Seed set to 42


### Checking for GPU Devices

In this step, we check for the availability of GPU devices and print the device currently being used by PyTorch. This ensures that the computations are performed on the most efficient hardware available.

In [5]:
# Check if CUDA is available and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('Torch Version: ', torch.__version__)
print('Using device: ', device)
if device.type == 'cuda':
    print('Cuda Version: ', torch.version.cuda)
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
    torch.set_float32_matmul_precision('high')

Torch Version:  2.7.0+cu128
Using device:  cuda
Cuda Version:  12.8
NVIDIA GeForce RTX 5060 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


### Defining Transformations and Instantiating DataModule

In this step, we will define the necessary data transformations and initialize the `Animal_DataModule` with the provided configuration.

In [6]:
# # TODO: Define transformations here

# # Example for transformation
# from torchvision import transforms
# transform = transforms.Compose([
#     transforms.Resize((300, 300)),  # Resize images to match EfficientNet input size
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard ImageNet normalization
# ])

# dl = Animal_DataModule(config['path_to_data'])

### Creating the Model

In this step, we will define the model architecture and print its summary using the `ModelSummary` utility from PyTorch Lightning. This provides an overview of the model's layers, parameters, and structure.

In [7]:
timm.list_models()

['aimv2_1b_patch14_224',
 'aimv2_1b_patch14_336',
 'aimv2_1b_patch14_448',
 'aimv2_3b_patch14_224',
 'aimv2_3b_patch14_336',
 'aimv2_3b_patch14_448',
 'aimv2_huge_patch14_224',
 'aimv2_huge_patch14_336',
 'aimv2_huge_patch14_448',
 'aimv2_large_patch14_224',
 'aimv2_large_patch14_336',
 'aimv2_large_patch14_448',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'beitv2_base_patch16_224',
 'beitv2_large_patch16_224',
 'botnet26t_256',
 'botnet50ts_256',
 'caformer_b36',
 'caformer_m36',
 'caformer_s18',
 'caformer_s36',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_medium',
 'coat_lite_medium_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_small',
 'coat_tiny',
 'coatnet_0_224',
 'coatnet_0_rw_224',
 'coa

### Transfer Learning with EfficientNet_B3

In this step, we utilize the EfficientNet_B3 model for transfer learning. The model is pre-trained on ImageNet, and we adapt it to our specific task by modifying the classifier layer to match the number of output classes (`num_classes`). 

We freeze all layers except the classifier to retain the pre-trained features while allowing the classifier to learn task-specific features. The model summary provides an overview of the architecture and the number of trainable parameters.

In [8]:
# Load pretrained model
model = timm.create_model(
    'efficientnet_b3',      # Hardcoded for now
    pretrained=True,
)
# Define number of classes and classifier
num_classes = 1             # Hardcoded for now, Dwarf Rabbit OK/NOK output    
model.classifier = torch.nn.Linear(model.classifier.in_features, num_classes)

# Freeze all layers except the classifier
for param in model.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True

# Wrap the model in the LightningModule
from models.model_transferlearning import TransferLearningModule
lightning_model = TransferLearningModule(model, config['learning_rate'])

### Data Preparation for EfficientNet_B3

To prepare the data for training with EfficientNet_B3, we define a set of image transformations that resize all images to 300x300 pixels, convert them to tensors, and normalize them using the standard ImageNet mean and standard deviation. These transformations ensure compatibility with the input requirements of the EfficientNet architecture.

We then instantiate the `BinaryCIFARDataModule` with the defined transformations, batch size, and number of workers from the configuration. After setup, we create the training, validation, and test data loaders. The sizes of each dataset split are printed for verification.

In [9]:
# Define transformations required for the used model
transform = transforms.Compose([
    transforms.Resize((300, 300)),  # Resize images to match EfficientNet input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard ImageNet normalization
])

dm = BinaryCIFARDataModule(transform=transform, batch_size=config['batch_size'], num_workers=2, persistent_workers=True)
dm.setup()
train_loader = dm.train_dataloader()
val_loader = dm.val_dataloader()
test_loader = dm.test_dataloader()

print('Train dataset size:', len(dm.train_dataset))
print('Validation dataset size:', len(dm.val_dataset))
print('Test dataset size:', len(dm.test_dataset))

TypeError: BinaryCIFARDataModule.__init__() got an unexpected keyword argument 'persistent_workers'

### Training and Logging with Weights & Biases

In this step, we initialize the Weights & Biases (wandb) logger to track experiment metrics, hyperparameters, and model checkpoints. The logger is configured with project and experiment names, as well as key training parameters such as dataset, batch size, maximum epochs, and learning rate.

We then set up the PyTorch Lightning `Trainer` with the wandb logger and an early stopping callback to monitor validation loss. The model is trained using the specified datamodule, and all relevant metrics are automatically logged to wandb for further analysis and visualization. After training, wandb logging is finalized to ensure all data is properly saved.

In [12]:
# Initialize the Wandb logger
wandb_logger = WandbLogger(
    project=config['wandb_project_name'],
    name=config['wandb_experiment_name'],
    config={
        'dataset': 'CIFAR-binary',
        'batch_size': config['batch_size'],
        'max_epochs': config['max_epochs'],
        'learning_rate': config['learning_rate']
    }
)

# Train the model and log relevant metrics using PyTorch Lightning Trainer and WandbLogger
trainer = Trainer(
    max_epochs=config['max_epochs'],
    default_root_dir='model/checkpoint/',
    accelerator="auto",
    devices="auto",
    strategy="auto",
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, mode='min')],
    logger=wandb_logger
)

trainer.fit(model=lightning_model, datamodule=dm)

# Finish wandb logging
wandb.finish()

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type              | Params | Mode 
-------------------------------------------------------------
0 | model          | EfficientNet      | 10.7 M | train
1 | criterion      | BCEWithLogitsLoss | 0      | train
2 | train_accuracy | BinaryAccuracy    | 0      | train
3 | val_accuracy   | BinaryAccuracy    | 0      | train
4 | val_precision  | BinaryPrecision   | 0      | train
5 | val_recall     | BinaryRecall      | 0      | train
-------------------------------------------------------------
1.5 K     Trainable params
10.7 M    Non-trainable params
10.7 M    Total params
42.791    Total estimated model params size (MB)
538       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryAccuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryPrecision was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric BinaryRecall was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
c:\Users\lukas\anaconda3\envs\VDKI-Projekt\Lib\site-packages\pytorch_lightning\trainer\connecto

Epoch 5:  32%|███▏      | 20/63 [00:16<00:35,  1.22it/s, v_num=9w8n, val_loss=0.363]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

### Transfer Learning with ResNet50D

In this step, we utilize the ResNet50D model for transfer learning. The model is pre-trained on ImageNet, and we adapt it to our specific task by modifying the fully connected (`fc`) layer to match the number of output classes (`num_classes`).

We freeze all layers except the `fc` layer to retain the pre-trained features while allowing the classifier to learn task-specific features. The model summary provides an overview of the architecture and the number of trainable parameters.

In [ ]:
# Load pretrained model
model = timm.create_model(
    'resnet50d',      # Hardcoded for now
    pretrained=True,
)
# Define number of classes and classifier
num_classes = 1             # Hardcoded for now, Dwarf Rabbit OK/NOK output    
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# Freeze all layers except the classifier
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

# Print model summary
summary(model, input_size=(1, 3, 224, 224), depth=2)

# Wrap the model in the LightningModule
from models.model_transferlearning import TransferLearningModule
lightning_model = TransferLearningModule(model, config['learning_rate'])

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 1]                    --
├─Sequential: 1-1                        [1, 64, 112, 112]         --
│    └─Conv2d: 2-1                       [1, 32, 112, 112]         (864)
│    └─BatchNorm2d: 2-2                  [1, 32, 112, 112]         (64)
│    └─ReLU: 2-3                         [1, 32, 112, 112]         --
│    └─Conv2d: 2-4                       [1, 32, 112, 112]         (9,216)
│    └─BatchNorm2d: 2-5                  [1, 32, 112, 112]         (64)
│    └─ReLU: 2-6                         [1, 32, 112, 112]         --
│    └─Conv2d: 2-7                       [1, 64, 112, 112]         (18,432)
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         (128)
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 256, 56, 56]       

### Data Preparation for ResNet50D

To prepare the data for training with ResNet50D, we define a set of image transformations that resize all images to 224x224 pixels, convert them to tensors, and normalize them using the standard ImageNet mean and standard deviation. These transformations ensure compatibility with the input requirements of the ResNet architecture.

We then instantiate the `BinaryCIFARDataModule` with the defined transformations, batch size, and number of workers from the configuration. After setup, we create the training, validation, and test data loaders. The sizes of each dataset split are printed for verification.

In [ ]:
# Define transformations required for the used model
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to match ResNet input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dm = BinaryCIFARDataModule(transform=transform, batch_size=config['batch_size'], num_workers=2, persistent_workers=True)
dm.setup()
train_loader = dm.train_dataloader()
val_loader = dm.val_dataloader()
test_loader = dm.test_dataloader()

print('Train dataset size:', len(dm.train_dataset))
print('Validation dataset size:', len(dm.val_dataset))
print('Test dataset size:', len(dm.test_dataset))

### Training and Logging with Weights & Biases

In this step, we initialize the Weights & Biases (wandb) logger to track experiment metrics, hyperparameters, and model checkpoints. The logger is configured with project and experiment names, as well as key training parameters such as dataset, batch size, maximum epochs, and learning rate.

We then set up the PyTorch Lightning `Trainer` with the wandb logger and an early stopping callback to monitor validation loss. The model is trained using the specified datamodule, and all relevant metrics are automatically logged to wandb for further analysis and visualization. After training, wandb logging is finalized to ensure all data is properly saved.

In [ ]:
# Initialize the Wandb logger
wandb_logger = WandbLogger(
    project=config['wandb_project_name'],
    name=config['wandb_experiment_name'],
    config={
        'dataset': 'CIFAR-binary',
        'batch_size': config['batch_size'],
        'max_epochs': config['max_epochs'],
        'learning_rate': config['learning_rate']
    }
)

# Train the model and log relevant metrics using PyTorch Lightning Trainer and WandbLogger
trainer = Trainer(
    max_epochs=config['max_epochs'],
    default_root_dir='model/checkpoint/',
    accelerator="auto",
    devices="auto",
    strategy="auto",
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, mode='min')],
    logger=wandb_logger
)

trainer.fit(model=lightning_model, datamodule=dm)

# Finish wandb logging
wandb.finish()

### Transfer Learning with InceptionV4

In this step, we utilize the InceptionV4 model for transfer learning. The model is pre-trained on ImageNet, and we adapt it to our specific task by modifying the `last_linear` layer to match the number of output classes (`num_classes`).

We freeze all layers except the `last_linear` layer to retain the pre-trained features while allowing the classifier to learn task-specific features. The model summary provides an overview of the architecture and the number of trainable parameters.

In [ ]:
# Load pretrained model
model = timm.create_model(
    'inception_v4',      # Hardcoded for now
    pretrained=True,
)
# Define number of classes and classifier
num_classes = 1             # Hardcoded for now, Dwarf Rabbit OK/NOK output    
model.last_linear = torch.nn.Linear(model.last_linear.in_features, num_classes)

# Freeze all layers except the classifier
for param in model.parameters():
    param.requires_grad = False
for param in model.last_linear.parameters():
    param.requires_grad = True

# Print model summary
summary(model, input_size=(1, 3, 299, 299), depth=2)

# Wrap the model in the LightningModule
from models.model_transferlearning import TransferLearningModule
lightning_model = TransferLearningModule(model, config['learning_rate'])

Layer (type:depth-idx)                             Output Shape              Param #
InceptionV4                                        [1, 1]                    --
├─Sequential: 1-1                                  [1, 1536, 8, 8]           --
│    └─ConvNormAct: 2-1                            [1, 32, 149, 149]         (928)
│    └─ConvNormAct: 2-2                            [1, 32, 147, 147]         (9,280)
│    └─ConvNormAct: 2-3                            [1, 64, 147, 147]         (18,560)
│    └─Mixed3a: 2-4                                [1, 160, 73, 73]          (55,488)
│    └─Mixed4a: 2-5                                [1, 192, 71, 71]          (189,312)
│    └─Mixed5a: 2-6                                [1, 384, 35, 35]          (332,160)
│    └─InceptionA: 2-7                             [1, 384, 35, 35]          (317,632)
│    └─InceptionA: 2-8                             [1, 384, 35, 35]          (317,632)
│    └─InceptionA: 2-9                             [1, 384, 35, 35]

### Data Preparation for InceptionV4

To prepare the data for training with InceptionV4, we define a set of image transformations that resize all images to 299x299 pixels, convert them to tensors, and normalize them using a mean and standard deviation of 0.5 for each channel. These transformations ensure compatibility with the input requirements of the InceptionV4 architecture.

We then instantiate the `BinaryCIFARDataModule` with the defined transformations, batch size, and number of workers from the configuration. After setup, we create the training, validation, and test data loaders. The sizes of each dataset split are printed for verification.

In [ ]:
# Define transformations required for the used model
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Resize images to match EfficientNet input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) # Standard ImageNet normalization
])

dm = BinaryCIFARDataModule(transform=transform, batch_size=config['batch_size'], num_workers=2, persistent_workers=True)
dm.setup()
train_loader = dm.train_dataloader()
val_loader = dm.val_dataloader()
test_loader = dm.test_dataloader()

print('Train dataset size:', len(dm.train_dataset))
print('Validation dataset size:', len(dm.val_dataset))
print('Test dataset size:', len(dm.test_dataset))

### Training and Logging with Weights & Biases

In this step, we initialize the Weights & Biases (wandb) logger to track experiment metrics, hyperparameters, and model checkpoints. The logger is configured with project and experiment names, as well as key training parameters such as dataset, batch size, maximum epochs, and learning rate.

We then set up the PyTorch Lightning `Trainer` with the wandb logger and an early stopping callback to monitor validation loss. The model is trained using the specified datamodule, and all relevant metrics are automatically logged to wandb for further analysis and visualization. After training, wandb logging is finalized to ensure all data is properly saved.

In [ ]:
# Initialize the Wandb logger
wandb_logger = WandbLogger(
    project=config['wandb_project_name'],
    name=config['wandb_experiment_name'],
    config={
        'dataset': 'CIFAR-binary',
        'batch_size': config['batch_size'],
        'max_epochs': config['max_epochs'],
        'learning_rate': config['learning_rate']
    }
)

# Train the model and log relevant metrics using PyTorch Lightning Trainer and WandbLogger
trainer = Trainer(
    max_epochs=config['max_epochs'],
    default_root_dir='model/checkpoint/',
    accelerator="auto",
    devices="auto",
    strategy="auto",
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, mode='min')],
    logger=wandb_logger
)

trainer.fit(model=lightning_model, datamodule=dm)

# Finish wandb logging
wandb.finish()